# LangChain ChormaDB

In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma

In [12]:
from langchain.schema import Document


doc1 = Document(
        page_content="Anakin Skywalker was a Jedi Knight who later became Darth Vader. Known for his strength in the Force and emotional volatility.",
        metadata={"duel_technique": "Aggressive Form V (Djem So) with raw power"}
    )
doc2 = Document(
        page_content="Obi-Wan Kenobi was a wise and disciplined Jedi Master, known for his patience and skill in defensive combat.",
        metadata={"duel_technique": "Form III (Soresu) – defensive and precise"}
    )
doc3 = Document(
        page_content="Yoda was a Grand Master of the Jedi Order, small in stature but immensely powerful and agile in combat.",
        metadata={"duel_technique": "Form IV (Ataru) – acrobatic and fast-paced"}
    )
doc4 = Document(
        page_content="Darth Maul was a Sith Lord with a fearsome double-bladed lightsaber and intense physical combat style.",
        metadata={"duel_technique": "Juyo – Form VII with aggressive acrobatics"}
    )
doc5 = Document(
        page_content="Mace Windu was a Jedi Master known for his unique mastery of Form VII, balancing aggression with inner control.",
        metadata={"duel_technique": "Vaapad – controlled Form VII variant"}
    )

In [14]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [15]:
vector_store = Chroma(
    embedding_function=OpenAIEmbeddings(),
    persist_directory='chroma-db', 
    collection_name='sample'
)

## Adding Documents to Vector Database

In [16]:
vector_store.add_documents(docs)

['0e61befc-3ff1-4dd6-a48d-6fd7ddd9e459',
 '74b2787a-cf1b-47e6-bc9a-4fd350d1031e',
 '4d85f22f-93cc-4233-9493-84e6bf281f56',
 'a7d21de1-d134-4a9b-b748-3f9a721a3f66',
 '6ae9c737-9c1b-4c88-8b67-79e4fcf9bdb5']

## View Documents

In [19]:
vector_store.get(include=['embeddings', 'documents', 'metadatas'])

{'ids': ['0e61befc-3ff1-4dd6-a48d-6fd7ddd9e459',
  '74b2787a-cf1b-47e6-bc9a-4fd350d1031e',
  '4d85f22f-93cc-4233-9493-84e6bf281f56',
  'a7d21de1-d134-4a9b-b748-3f9a721a3f66',
  '6ae9c737-9c1b-4c88-8b67-79e4fcf9bdb5'],
 'embeddings': array([[-0.01214767, -0.0328187 , -0.00651125, ..., -0.00197306,
         -0.00279946,  0.00174654],
        [-0.00494969, -0.02309853, -0.0138845 , ..., -0.01031184,
         -0.0004676 , -0.00720877],
        [-0.00715249, -0.01840835, -0.00640126, ..., -0.00961451,
         -0.00279818, -0.03123612],
        [-0.0235865 ,  0.00780861, -0.00197786, ..., -0.00681888,
          0.00233294,  0.00032877],
        [-0.00729117, -0.0104587 , -0.00489764, ..., -0.00435779,
          0.00524236, -0.02976306]], shape=(5, 1536)),
 'documents': ['Anakin Skywalker was a Jedi Knight who later became Darth Vader. Known for his strength in the Force and emotional volatility.',
  'Obi-Wan Kenobi was a wise and disciplined Jedi Master, known for his patience and skill in 

## Search Documents 

In [20]:
vector_store.similarity_search(
    query="Who was the best duelist?",
    k=2  # How many similar results you want to show
)

[Document(id='a7d21de1-d134-4a9b-b748-3f9a721a3f66', metadata={'duel_technique': 'Juyo – Form VII with aggressive acrobatics'}, page_content='Darth Maul was a Sith Lord with a fearsome double-bladed lightsaber and intense physical combat style.'),
 Document(id='74b2787a-cf1b-47e6-bc9a-4fd350d1031e', metadata={'duel_technique': 'Form III (Soresu) – defensive and precise'}, page_content='Obi-Wan Kenobi was a wise and disciplined Jedi Master, known for his patience and skill in defensive combat.')]

## Semantic Search with score 

In [22]:
vector_store.similarity_search_with_score(
    query="Who turned into dark side?",
    k=2
)

[(Document(id='0e61befc-3ff1-4dd6-a48d-6fd7ddd9e459', metadata={'duel_technique': 'Aggressive Form V (Djem So) with raw power'}, page_content='Anakin Skywalker was a Jedi Knight who later became Darth Vader. Known for his strength in the Force and emotional volatility.'),
  0.3341638147830963),
 (Document(id='a7d21de1-d134-4a9b-b748-3f9a721a3f66', metadata={'duel_technique': 'Juyo – Form VII with aggressive acrobatics'}, page_content='Darth Maul was a Sith Lord with a fearsome double-bladed lightsaber and intense physical combat style.'),
  0.3591403067111969)]

## Meta Data Filtering

In [25]:
vector_store.similarity_search_with_score(
    query="",
    filter={"duel_technique": "Vaapad – controlled Form VII variant"}
)

[(Document(id='6ae9c737-9c1b-4c88-8b67-79e4fcf9bdb5', metadata={'duel_technique': 'Vaapad – controlled Form VII variant'}, page_content='Mace Windu was a Jedi Master known for his unique mastery of Form VII, balancing aggression with inner control.'),
  0.6077194809913635)]

## Update Document

In [28]:
updated_doc1= Document(
    page_content="Anakin Skywalker was a Jedi Knight who later became Darth Vader was denied the Jedi Master title and he didn't even had high ground. This caused him to lose to ObiWan",
    metadata={"duel_technique": "Aggressive Form V (Djem So) with raw power"}
)

In [29]:
vector_store.update_document(document_id='0e61befc-3ff1-4dd6-a48d-6fd7ddd9e459', document=updated_doc1)

In [30]:
vector_store.get(include=['embeddings', 'documents'])

{'ids': ['0e61befc-3ff1-4dd6-a48d-6fd7ddd9e459',
  '74b2787a-cf1b-47e6-bc9a-4fd350d1031e',
  '4d85f22f-93cc-4233-9493-84e6bf281f56',
  'a7d21de1-d134-4a9b-b748-3f9a721a3f66',
  '6ae9c737-9c1b-4c88-8b67-79e4fcf9bdb5'],
 'embeddings': array([[-0.00603055, -0.03562185, -0.00915714, ...,  0.00107688,
         -0.01919192, -0.00142213],
        [-0.00494969, -0.02309853, -0.0138845 , ..., -0.01031184,
         -0.0004676 , -0.00720877],
        [-0.00715249, -0.01840835, -0.00640126, ..., -0.00961451,
         -0.00279818, -0.03123612],
        [-0.0235865 ,  0.00780861, -0.00197786, ..., -0.00681888,
          0.00233294,  0.00032877],
        [-0.00729117, -0.0104587 , -0.00489764, ..., -0.00435779,
          0.00524236, -0.02976306]], shape=(5, 1536)),
 'documents': ["Anakin Skywalker was a Jedi Knight who later became Darth Vader was denied the Jedi Master title and he didn't even had high ground. This caused him to lose to ObiWan",
  'Obi-Wan Kenobi was a wise and disciplined Jedi Mast

## Delete Document

In [32]:
vector_store.delete(ids=["6ae9c737-9c1b-4c88-8b67-79e4fcf9bdb5"])

In [33]:
vector_store.get(include=['documents'])

{'ids': ['0e61befc-3ff1-4dd6-a48d-6fd7ddd9e459',
  '74b2787a-cf1b-47e6-bc9a-4fd350d1031e',
  '4d85f22f-93cc-4233-9493-84e6bf281f56',
  'a7d21de1-d134-4a9b-b748-3f9a721a3f66'],
 'embeddings': None,
 'documents': ["Anakin Skywalker was a Jedi Knight who later became Darth Vader was denied the Jedi Master title and he didn't even had high ground. This caused him to lose to ObiWan",
  'Obi-Wan Kenobi was a wise and disciplined Jedi Master, known for his patience and skill in defensive combat.',
  'Yoda was a Grand Master of the Jedi Order, small in stature but immensely powerful and agile in combat.',
  'Darth Maul was a Sith Lord with a fearsome double-bladed lightsaber and intense physical combat style.'],
 'uris': None,
 'included': ['documents'],
 'data': None,
 'metadatas': None}